In [17]:
########################start##############################
import pandas as pd
df=pd.read_csv('./datas/Data_new.csv',index_col=0)
df.index=pd.to_datetime(df.index,format='%Y-%m-%d') 
df_train=df.loc['2020-01-07':'2021-04-02'] 
df_test=df.loc['2021-04-02':'2021-04-23']

name=['cityin','cityto','incity','city']
name1=['cityin','cityto','incity']
x_train=df_train.drop(name,axis=1).values  
y_train=df_train[name1].values
x_test=df_test.drop(name,axis=1).values  
y_test=df_test[name1].values
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

#####################标准化##############################
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(x_train)
x_train = ss.transform(x_train) 
x_test = ss.transform(x_test)

(69056, 43) (7942, 43) (69056, 3) (7942, 3)


In [18]:
import csv
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import joblib
plt.rcParams['font.sans-serif'] = ['SimHei']  # 中文字体设置-黑体
plt.rcParams['axes.unicode_minus'] = False  # 解决保存图像是负号'-'显示为方块的问题
sns.set(font='SimHei',font_scale=1.5)  # 解决Seaborn中文显示问题并调整字体大小
sns.set_style("white")
def mkdir(path):
    folder = os.path.exists(path)
    if not folder:                   
        os.makedirs(path)            
        print("---  new folder...  ---")
        print("---  OK  ---")
    else:
        print("---  There is this folder!  ---")
mkdir("./results/")      
mkdir('./draw_picture/pred_result/')
from sklearn.metrics import mean_absolute_error,mean_squared_error,median_absolute_error,r2_score,mean_squared_log_error

def calculate(y_true, y_predict, n, p):
    y_true = y_true
    y_predict = y_predict
    mse = mean_squared_error(y_true, y_predict)
    rmse = np.sqrt(mean_squared_error(y_true, y_predict))
    mae = mean_absolute_error(y_true, y_predict)
    r2 = r2_score(y_true, y_predict)
    mad = median_absolute_error(y_true, y_predict)
    mape = np.mean(np.abs((y_true - y_predict) / y_true)) * 100
    r2_adjusted = 1-((1-r2)*(n-1))/(n-p-1)
    # print('MSE: ', mse)
    # print('RMSE: ', rmse)
    # print('MAE: ', mae)
    # print('R2: ', r2)
    # print('MAD:', mad)
    # print('MAPE:', mape)
    # print('R2_Adjusted: ',r2_adjusted)
    return mse,rmse,mae,r2,mad,mape,r2_adjusted
# 将参数和评估结果写入文件
def write_csv_result(path_1,path_2,all_metrics,all_parameter):
    with open(path_1,"a",encoding="utf-8",newline="")as f:
        f = csv.writer(f)
        f.writerow(all_metrics)
    with open(path_2,"a",encoding="utf-8",newline="")as f:
        f = csv.writer(f)
        f.writerow(all_parameter)


---  There is this folder!  ---
---  There is this folder!  ---


In [66]:
def Mlp(parameter,train_x=x_train,train_y=y_train,test_x=x_test,test_y=y_test,df_test=df_test):
    city=df_test['city'].tolist()
#     date=df_test.index.strftime('%Y-%m-%d').tolist()
    date=(df_test.index+datetime.timedelta(days=7)).strftime('%Y-%m-%d').tolist()
    m=parameter[0]
    t=parameter[1]
    l=parameter[2]
    hd=eval(parameter[3])
    model = MLPRegressor(hidden_layer_sizes=hd, activation="relu",
                                         solver='adam', alpha=0.0001,
                                         batch_size='auto', learning_rate="constant",
                                         learning_rate_init=l,
                                         power_t=0.5, max_iter=m,tol=t)

    wrapper = RegressorChain(model)
    wrapper.fit(train_x,train_y)
    pred_test = wrapper.predict(test_x)
    #   

    y_test=test_y       
    y_test1=[]
    for i in y_test:
        y_test1.append(i[0])
    pred_test1=[]
    for i in pred_test:
        pred_test1.append(i[0])
    y_test2=[]
    for i in y_test:
        y_test2.append(i[1])
    pred_test2=[]
    for i in pred_test:
        pred_test2.append(i[1])
    y_test3=[]
    for i in y_test:
        y_test3.append(i[2])
    pred_test3=[]
    for i in pred_test:
        pred_test3.append(i[2])
    pred_test1 = np.array(pred_test1).reshape(-1,1)
    y_test1 = np.array(y_test1).reshape(-1,1)
    sample_n = pred_test1.shape[0]
    feature_n = x_test.shape[1]
    mse1,rmse1,mae1,r21,mad1,mape1,r2_adjusted1 = calculate(y_test1,pred_test1,sample_n,feature_n)


    pred_test2 = np.array(pred_test2).reshape(-1,1)
    y_test2 = np.array(y_test2).reshape(-1,1)
    sample_n = pred_test2.shape[0]
    feature_n = x_test.shape[1]
    mse2,rmse2,mae2,r22,mad2,mape2,r2_adjusted2 = calculate(y_test2,pred_test2,sample_n,feature_n)


    pred_test3 = np.array(pred_test3).reshape(-1,1)
    y_test3 = np.array(y_test3).reshape(-1,1)
    sample_n = pred_test3.shape[0]
    feature_n = x_test.shape[1]
    mse3,rmse3,mae3,r23,mad3,mape3,r2_adjusted3 = calculate(y_test3,pred_test3,sample_n,feature_n)

    mse,rmse,mae,r2,mad,mape,r2_adjusted = (mse1+mse2+mse3)/3,(rmse1+rmse2+rmse3)/3,(mae1+mae2+mae3)/3,(r21+r22+r23)/3,(mad1+mad2+mad3)/3,(mape1+mape2+mape3)/3,(r2_adjusted1+r2_adjusted2+r2_adjusted3)/3

    df=pd.DataFrame([date,pred_test1.flatten(),y_test1.flatten(),pred_test2.flatten(),y_test2.flatten(),pred_test3.flatten(),y_test3.flatten(),city]).T
    df.columns=['date','cityin_pred','cityin_true','cityto_pred','cityto_true','incity_pred','incity_true','city']
    df.to_csv('./result/Mlp_Result.csv',index=0,encoding='gbk')
    print(mse,rmse,mae,r2,mad,mape,r2_adjusted)
    return df

In [67]:
def Knn(parameter,train_x=x_train,train_y=y_train,test_x=x_test,test_y=y_test,df_test=df_test):
    city=df_test['city'].tolist()
#     date=df_test.index.strftime('%Y-%m-%d').tolist()
    date=(df_test.index+datetime.timedelta(days=7)).strftime('%Y-%m-%d').tolist()
    m=parameter[0]
    a=parameter[1]
    w=parameter[2]
    n=parameter[3]
    l=parameter[4]
    
    model = KNeighborsRegressor(n_neighbors=n,leaf_size=l,weights=w,metric=m,algorithm=a)

    wrapper = RegressorChain(model)
    wrapper.fit(train_x,train_y)
    pred_test = wrapper.predict(test_x)
    #   

    y_test=test_y       
    y_test1=[]
    for i in y_test:
        y_test1.append(i[0])
    pred_test1=[]
    for i in pred_test:
        pred_test1.append(i[0])
    y_test2=[]
    for i in y_test:
        y_test2.append(i[1])
    pred_test2=[]
    for i in pred_test:
        pred_test2.append(i[1])
    y_test3=[]
    for i in y_test:
        y_test3.append(i[2])
    pred_test3=[]
    for i in pred_test:
        pred_test3.append(i[2])
    pred_test1 = np.array(pred_test1).reshape(-1,1)
    y_test1 = np.array(y_test1).reshape(-1,1)
    sample_n = pred_test1.shape[0]
    feature_n = x_test.shape[1]
    mse1,rmse1,mae1,r21,mad1,mape1,r2_adjusted1 = calculate(y_test1,pred_test1,sample_n,feature_n)


    pred_test2 = np.array(pred_test2).reshape(-1,1)
    y_test2 = np.array(y_test2).reshape(-1,1)
    sample_n = pred_test2.shape[0]
    feature_n = x_test.shape[1]
    mse2,rmse2,mae2,r22,mad2,mape2,r2_adjusted2 = calculate(y_test2,pred_test2,sample_n,feature_n)


    pred_test3 = np.array(pred_test3).reshape(-1,1)
    y_test3 = np.array(y_test3).reshape(-1,1)
    sample_n = pred_test3.shape[0]
    feature_n = x_test.shape[1]
    mse3,rmse3,mae3,r23,mad3,mape3,r2_adjusted3 = calculate(y_test3,pred_test3,sample_n,feature_n)

    mse,rmse,mae,r2,mad,mape,r2_adjusted = (mse1+mse2+mse3)/3,(rmse1+rmse2+rmse3)/3,(mae1+mae2+mae3)/3,(r21+r22+r23)/3,(mad1+mad2+mad3)/3,(mape1+mape2+mape3)/3,(r2_adjusted1+r2_adjusted2+r2_adjusted3)/3

    df=pd.DataFrame([date,pred_test1.flatten(),y_test1.flatten(),pred_test2.flatten(),y_test2.flatten(),pred_test3.flatten(),y_test3.flatten(),city]).T
    df.columns=['date','cityin_pred','cityin_true','cityto_pred','cityto_true','incity_pred','incity_true','city']
    df.to_csv('./result/Knn_Result.csv',index=0,encoding='gbk')
    print(mse,rmse,mae,r2,mad,mape,r2_adjusted)
    return df

In [19]:
from sklearn.ensemble import GradientBoostingRegressor
import csv
import os
from sklearn.multioutput import RegressorChain
def Gbdt(parameter,train_x=x_train,train_y=y_train,test_x=x_test,test_y=y_test,df_test=df_test):
    city=df_test['city'].tolist()
#     date=df_test.index.strftime('%Y-%m-%d').tolist()
    date=(df_test.index+datetime.timedelta(days=7)).strftime('%Y-%m-%d').tolist()
    n=parameter[0]
    l=parameter[1]
    lo=parameter[2]
    sub=parameter[3]
    mi=parameter[4]
    ma=parameter[5]
    ms=parameter[6]
    model = GradientBoostingRegressor(random_state=17,n_estimators=n,learning_rate=l,loss=lo,subsample=sub,max_depth=ma,min_samples_split=mi,min_samples_leaf=ms)

    wrapper = RegressorChain(model)
    wrapper.fit(train_x,train_y)
    pred_test = wrapper.predict(test_x)
    #   

    y_test=test_y       
    y_test1=[]
    for i in y_test:
        y_test1.append(i[0])
    pred_test1=[]
    for i in pred_test:
        pred_test1.append(i[0])
    y_test2=[]
    for i in y_test:
        y_test2.append(i[1])
    pred_test2=[]
    for i in pred_test:
        pred_test2.append(i[1])
    y_test3=[]
    for i in y_test:
        y_test3.append(i[2])
    pred_test3=[]
    for i in pred_test:
        pred_test3.append(i[2])
    pred_test1 = np.array(pred_test1).reshape(-1,1)
    y_test1 = np.array(y_test1).reshape(-1,1)
    sample_n = pred_test1.shape[0]
    feature_n = x_test.shape[1]
    mse1,rmse1,mae1,r21,mad1,mape1,r2_adjusted1 = calculate(y_test1,pred_test1,sample_n,feature_n)


    pred_test2 = np.array(pred_test2).reshape(-1,1)
    y_test2 = np.array(y_test2).reshape(-1,1)
    sample_n = pred_test2.shape[0]
    feature_n = x_test.shape[1]
    mse2,rmse2,mae2,r22,mad2,mape2,r2_adjusted2 = calculate(y_test2,pred_test2,sample_n,feature_n)


    pred_test3 = np.array(pred_test3).reshape(-1,1)
    y_test3 = np.array(y_test3).reshape(-1,1)
    sample_n = pred_test3.shape[0]
    feature_n = x_test.shape[1]
    mse3,rmse3,mae3,r23,mad3,mape3,r2_adjusted3 = calculate(y_test3,pred_test3,sample_n,feature_n)

    mse,rmse,mae,r2,mad,mape,r2_adjusted = (mse1+mse2+mse3)/3,(rmse1+rmse2+rmse3)/3,(mae1+mae2+mae3)/3,(r21+r22+r23)/3,(mad1+mad2+mad3)/3,(mape1+mape2+mape3)/3,(r2_adjusted1+r2_adjusted2+r2_adjusted3)/3

    df=pd.DataFrame([date,pred_test1.flatten(),y_test1.flatten(),pred_test2.flatten(),y_test2.flatten(),pred_test3.flatten(),y_test3.flatten(),city]).T
    df.columns=['date','cityin_pred','cityin_true','cityto_pred','cityto_true','incity_pred','incity_true','city']
    df.to_csv('./result/Gbdt_Result.csv',index=0,encoding='gbk')
    print(mse,rmse,mae,r2,mad,mape,r2_adjusted)
    return df

In [20]:
def pred_result(cityname='武汉',aim='cityin',df=df,name=i):
    fig = plt.figure(figsize=(13, 5))
    ax = fig.add_subplot(111)

    color_sequence = ['#1f77b4', '#aec7e8', '#ff7f0e', '#ffbb40', '#2ca02c',
                    '#98df8a', '#d62728', '#ff9896', '#9467bd', '#c5b0d5',
                    '#8c564b', '#c49c94', '#e377c2', '#f7b6d2', '#7f7f7f',
                    '#c7c7c7', '#bcbd22', '#dbdb8d', '#17becf', '#9edae5']
    xtick = 10 # 显示10个x轴坐标
    date=df[df['city']==cityname]['date'].tolist()
    l = len(date) # 数据长度
    p = round(l / xtick) # x轴数据间隔

    ax.set_xticks(range(0, l, p))  
    ax.set_xticklabels([d.replace('2021-','') for d in date[::p]],fontsize=15)
    ax.plot(date, df[df['city']==cityname][aim+'_pred'].tolist(), '-o',label="2021_pred", color=color_sequence[1], linewidth=2.5,markersize=6)
    ax.plot(date, df[df['city']==cityname][aim+'_true'].tolist(), '-o',label="2021_true", color=color_sequence[7], linewidth=2.5,markersize=6)

    # #降低轴显示密度，达到美观目的
    # plt.gca().xaxis.set_major_locator(ticker.MultipleLocator(7))
    ax.grid(alpha=.3)

    ax.set_ylabel('Activity Intensity',fontsize=20)
    # ax.set_xlabel('Date',fontsize=15)
    plt.rc('ytick',labelsize=19)

    plt.gca().spines["top"].set_alpha(0.0)    

    plt.gca().spines["right"].set_alpha(0.0) 
    plt.tight_layout()

    plt.axis('tight')
#     ax.fill_between(date[26:29], y1=0, y2=1000, label='Qing Ming Festival', alpha=0.2, color=color_sequence[3], linewidth=1)
    ax.legend(fontsize =14)
    plt.savefig('./draw_picture/pred_result/'+name+cityname+'_'+aim+'_line.jpg')
    # plt.show()
    plt.close()

In [26]:
import csv
import os
import numpy as np
import datetime
from sklearn.multioutput import RegressorChain
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from itertools import combinations
from sklearn.neural_network import MLPRegressor
import matplotlib.pyplot as plt

name=['Gbdt']
for i in name:
    path_best='./results/{}_Migration_best.csv'.format(i)
    df=pd.read_csv(path_best)
    num=df.iloc[0,0]
    path_parameter='./results/{}_Migration_parameter.csv'.format(i)
    df=pd.read_csv(path_parameter,index_col=0)
    parameter=df[df.index==num].values[0]
    if i=='Knn':
        df=Knn(parameter,train_x=x_train,train_y=y_train,test_x=x_test,test_y=y_test,df_test=df_test)
        pred_result(cityname='武汉',aim='cityin',df=df,name=i)
    if i=='Mlp':
        df=Mlp(parameter,train_x=x_train,train_y=y_train,test_x=x_test,test_y=y_test,df_test=df_test)
        pred_result(cityname='武汉',aim='cityin',df=df,name=i)
    if i=='Gbdt':
        df=Gbdt(parameter,train_x=x_train,train_y=y_train,test_x=x_test,test_y=y_test,df_test=df_test)
        pred_result(cityname='武汉',aim='cityin',df=df,name=i)
    

0.07141906432754923 0.26653162327434526 0.14845885404585943 0.927041631743874 0.08035859453746995 12.828733492941176 0.9266444160139407


In [14]:
#########################各指标对比########################
import pandas as pd
a=pd.read_csv('./result/cityin.csv')
b=pd.read_csv('./result/cityin1.csv')
# a=a.loc[[0,1,2,4,5,6],:]
#绘制柱状图   
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = ['sans-serif']
plt.rcParams['font.sans-serif'] = ['SimHei']
if not os.path.exists('./pictures/model_result'):
    os.mkdir('./pictures/model_result')
for i in ['mse','rmse','mae','r2','mad','mape','r2_adjusted']:
    plt.figure(figsize=(12,8))
    X=np.arange(len(b['Unnamed: 0']))
    Y=np.array(a[i])
    Y1=np.array(b[i])
    bar_width = 0.25
    tick_label = a['Unnamed: 0'].tolist()

    # #显示每个柱的具体高度
    # for x,y in zip(X,Y):
    #     plt.text(x+0.0025,y,'%.2f' %y, ha='center',va='bottom')

    # for x,y1 in zip(X,Y1):
    #     plt.text(x+0.25,y1+0.01,'%.2f' %y1, ha='center',va='bottom')

    #绘制柱状图    
    plt.bar(X, Y, bar_width, align="center", color="blue", label="IM_ModelResult", alpha=0.5)
    plt.bar(X+bar_width, Y1, bar_width, color="red", align="center", \
            label="IM and COVID_ModelResult", alpha=0.5)

    plt.xlabel("model")
    plt.ylabel(i)
    # plt.title('Picture Name')

    plt.xticks(X+bar_width/2, tick_label)
    #显示图例
    plt.legend()
#     plt.show()
    plt.savefig('./pictures/model_result/'+i)
    plt.close()